In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from plotly.graph_objects import FigureWidget, Scatter
from plotly.figure_factory import create_quiver
from ipywidgets import interact as _original_interact, HBox, VBox, Layout, FloatSlider

In [2]:
import plotly.graph_objects
mydefault = plotly.graph_objects.layout.Template()
mydefault.layout.margin = dict(t=40, b=40, l=40, r=40)

# For 2D plots, turn off the grid, but keep the bounding lines
mydefault.layout.xaxis.showgrid = False
mydefault.layout.yaxis.showgrid = False
mydefault.layout.xaxis.showline = True
mydefault.layout.yaxis.showline = True

# Change default drag mode from zoom to pan, and do nothing on hover
mydefault.layout.dragmode = "pan"
mydefault.layout.hovermode = False
mydefault.layout.scene.hovermode = False

# Turn off spikes (hover lines that go from plot points to axes)
mydefault.layout.xaxis.showspikes = False
mydefault.layout.yaxis.showspikes = False
mydefault.layout.scene.xaxis.showspikes = False
mydefault.layout.scene.yaxis.showspikes = False
mydefault.layout.scene.zaxis.showspikes = False

# Turn off certain default features for specific types of plots
mydefault.data.contour = [plotly.graph_objects.Contour(
        showscale=False, 
)]
mydefault.data.scatter3d = [plotly.graph_objects.Scatter3d(
        hoverinfo="skip", 
)]
mydefault.data.surface = [plotly.graph_objects.Surface(
        showscale=False, 
        hoverinfo="skip", 
        contours_x_highlight=False, 
        contours_y_highlight=False, 
        contours_z_highlight=False, 
)]
mydefault.data.isosurface = [plotly.graph_objects.Isosurface(
        showscale=False, 
        hoverinfo="skip", 
)]
mydefault.data.cone = [plotly.graph_objects.Cone(
        showscale=False, 
        hoverinfo="skip", 
)]

# Install our template as the default
plotly.io.templates["mydefault"] = mydefault
plotly.io.templates.default = "mydefault"


In [3]:
def interact(_function_to_wrap=None, _layout="horizontal", **kwargs):
    """interact, but with widgets laid out in a horizontal flexbox layout

    This function works exactly like 'interact' (from ipywidgets), 
    except that instead of putting all of the widgets into a vertical box 
    (VBox), it uses a horizontal box (HBox) by default. The HBox uses a flexbox 
    layout, so that if there are many widgets, they'll wrap onto a second row. 
    
    Options:
        '_layout' - 'horizontal' by default. Anything else, and it will revert 
        back to using the default layout of 'interact' (a VBox). 
    """
    def decorator(f):
        retval = _original_interact(f, **kwargs)
        if _layout == "horizontal":
            widgets = retval.widget.children[:-1]
            output = retval.widget.children[-1]
            hbox = HBox(widgets, layout=Layout(flex_flow="row wrap"))
            retval.widget.children = (hbox, output)
        return retval
    if _function_to_wrap is None:
        # No function passed in, so this function must *return* a decorator
        return decorator
    # This function was called directly, *or* was used as a decorator directly
    return decorator(_function_to_wrap)


In [4]:
def latex_number(x, format="", suffix=""):
    if x == 0:
        return f"0 {suffix}"
    dtype = type(x)
    if dtype is complex or dtype is np.complex128:
        if x.imag > 0:
            return f"{x.real:{format}} + {x.imag:{format}}i {suffix}"
        if x.imag < 0:
            return f"{x.real:{format}} - {-x.imag:{format}}i {suffix}"
        return f"{x.real:{format}} {suffix}"
    if dtype is str or dtype is np.str_:
        return fr"\text{{{x}{suffix}}}"
    return f"{x:{format}} {suffix}"

def latex_vector(vector, **options):
    components = [latex_number(x, **options) for x in vector]
    return r"\begin{bmatrix} " + r" \\ ".join(components) + r" \end{bmatrix}"

def latex_matrix(matrix, **options):
    rows = [[latex_number(x, **options) for x in row] for row in matrix]
    rows = [" & ".join(row) for row in rows]
    return r"\begin{pmatrix} " + r" \\ ".join(rows) + r" \end{pmatrix}"

In [6]:
def linear_interactive():
    xmin, ymin, xmax, ymax = -10, -10, 10, 10
    t_range = np.linspace(0, 100, 5001)
    eigenlines = [Scatter(), Scatter()]
    figure = FigureWidget()
    figure.layout.title.text = "Phase portrait"
    figure.layout.xaxis.update(range=(xmin, xmax), title_text=r"$x$", constrain="domain")
    figure.layout.yaxis.update(range=(ymin, ymax), title_text=r"$y$", constrain="domain", 
                               scaleanchor="x", scaleratio=1)
    figure.add_scatter(mode="lines", line_color="limegreen", name="Vector field")
    vectorfield = figure.data[-1]
    figure.add_scatter(mode="lines", name="Eigenvector line", 
            visible="legendonly", legendgroup="Eigenlines")
    eigenline1 = figure.data[-1]
    figure.add_scatter(mode="lines", name="Eigenvector line", 
            visible="legendonly", legendgroup="Eigenlines")
    eigenline2 = figure.data[-1]
    figure.add_scatter(mode="lines", line_color="blue", line_smoothing=1.3, 
            name="Trajectory", visible="legendonly")
    trajectory = figure.data[-1]
    figure.add_annotation(x=0.85, y=0.5, xref="paper", yref="paper", 
            xanchor="left", yanchor="middle")

    figure2 = FigureWidget()
    figure2.layout.title.text = "Time series"
    figure2.layout.xaxis.update(range=(0, 30), title_text=r"$t$")
    figure2.layout.yaxis.update(range=(xmin, xmax), title_text=r"$x, \quad y$")
    figure2.add_scatter(mode="lines", line_smoothing=1.3, name="$x$")
    figure2.add_scatter(mode="lines", line_smoothing=1.3, name="$y$")
    x_timeseries, y_timeseries = figure2.data

    @interact(a=FloatSlider(1, min=-2, max=2, step=0.1, description=r"$a$"), 
              b=FloatSlider(0, min=-2, max=2, step=0.1, description=r"$b$"), 
              c=FloatSlider(0, min=-2, max=2, step=0.1, description=r"$c$"), 
              d=FloatSlider(1, min=-2, max=2, step=0.1, description=r"$d$"))
    def update(a, b, c, d):
        M = np.array([[a, b], [c, d]])
        x = np.linspace(xmin, xmax, 23)[1::2]
        y = np.linspace(ymin, ymax, 23)[1::2]
        x, y = np.meshgrid(x, y)
        u, v = (M @ np.array((x.flatten(), y.flatten()))).reshape(2, 11, 11)
        quiver = create_quiver(x, y, u, v).data[0]

        initial_state = np.array([0, 0], dtype=float)
        evalues, evectors = np.linalg.eig(M)
        for evalue, evector, line in zip(evalues, evectors.transpose(), eigenlines):
            if evalue.imag != 0:
                eigenlines[0].update(x=[0], y=[0])
                eigenlines[1].update(x=[0], y=[0])
                initial_state = (0.5, 0.5) if evalue.real > 0 else (9, 9) if evalue.real < 0 else (5,5)
                break
            initial_state += evector * (0.5 if evalue > 0 else 14 if evalue < 0 else 5)
            line.x, line.y = np.array([1.5*xmin*evector, 1.5*xmax*evector]).transpose()
            line.line.color = "red" if evalue > 0 else "green" if evalue < 0 else "darkorange"

        def F(t, x):
            return M @ x
        solution = solve_ivp(F, t_range[[0,-1]], initial_state, t_eval=t_range)
        evalue1, evalue2 = [latex_number(l, format=".2f") for l in evalues]
        label = fr"\text{{Matrix:}} \\ {latex_matrix(M, format='0.1f')} \\[24pt]"
        label += fr"\text{{Eigenvalues: }} \\ {evalue1} \text{{ and }} {evalue2}"

        with figure.batch_update():
            vectorfield.update(x=quiver.x, y=quiver.y)
            eigenline1.update(eigenlines[0])
            eigenline2.update(eigenlines[1])
            trajectory.update(x=solution.y[0], y=solution.y[1])
            figure.layout.annotations[0].update(text=f"${label}$")

        with figure2.batch_update():
            x_timeseries.update(x=t_range, y=solution.y[0])
            y_timeseries.update(x=t_range, y=solution.y[1])

    return VBox((figure, figure2))

linear_interactive()

interactive(children=(FloatSlider(value=1.0, description='$a$', max=2.0, min=-2.0), FloatSlider(value=0.0, des…

    'data': [{'line': {'color': 'limegreen'},
              'mode': 'lines',
   …